# Verification: CRPS and CRPSS for Unet baseline 

In [13]:
import os
import sys
import time
import numpy as np
from glob import glob
from datetime import datetime, timedelta
from scipy.interpolate import RegularGridInterpolator

import h5py

In [14]:
import numba as nb

In [15]:
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/')
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/libs/')

from namelist import *
import data_utils as du
import verif_utils as vu

In [16]:
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
with h5py.File(save_dir+'CNN_domain.hdf', 'r') as h5io:
    lon_01 = h5io['lon_01'][...]
    lat_01 = h5io['lat_01'][...]
    lon_GFS = h5io['lon_GFS'][...]
    lat_GFS = h5io['lat_GFS'][...]

In [18]:
name_output = '/glade/campaign/cisl/aiml/ksha/LDM_results/Baseline_2023_ini00_lead{:02d}.hdf'

**CRPS climatology reference**

In [19]:
with h5py.File(save_dir+'ERA5_CDFs.hdf', 'r') as h5io:
    ERA5_CDFs = h5io['ERA5_CDFs'][...]
    indx_in_GFS = h5io['indx_in_GFS'][...]
    indy_in_GFS = h5io['indy_in_GFS'][...]
    
ERA5_CDFs = 6*ERA5_CDFs # mm/hour to mm/6-hour

In [20]:
with h5py.File(name_output.format(3)) as h5io:
    MRMS_TRUE = h5io['MRMS_TRUE'][...]
    MRMS_PRED = h5io['MRMS_PRED'][...]

**CRPS from Unet outputs**

In [ ]:
N_days = 300
grid_shape = lon_01.shape # MRMS grid shape
LEADs = [6, 12, 18, 24]

CRPS_PRED_lead = np.empty((N_days,)+grid_shape); CRPS_PRED[...] = np.nan

MRMS_TRUE_6h_flat = np.empty((300, 256*576))
MRMS_PRED_6h_flat = np.empty((300, 10, 256*576))

for l, lead in enumerate(LEADs):

    MRMS_TRUE_6h_flat[...] = np.nan
    MRMS_PRED_6h_flat[...] = np.nan
    
    for lead_half in [lead-3, lead]:
        
        with h5py.File(name_output.format(lead)) as h5io:
            MRMS_TRUE = h5io['MRMS_TRUE'][...]
            MRMS_PRED = h5io['MRMS_PRED'][...]

        MRMS_TRUE_6h_flat += MRMS_TRUE.reshape(300, 256*576)
        MRMS_PRED_6h_flat += MRMS_PRED.reshape(300, 10, 256*576)

    # handle NaNs
    for i in range(300):
        for k in range(256*576):
            if (np.sum(np.isnan(MRMS_PRED_flat[i, :, k])) > 0) and (np.isnan(MRMS_TRUE_flat[i, k]) is False):
                MRMS_PRED_flat[i, :, k] = 0

    CRPS_pred, _, _ = vu.CRPS_1d_nan(MRMS_TRUE_flat, MRMS_PRED_flat)
    CRPS_PRED[:, l, ...] = CRPS_pred.reshape((300, 256, 576))
    
    # backup every lead time  
    tuple_save = (CRPS_PRED,)
    label_save = ['CRPS_PRED',]
    du.save_hdf5(tuple_save, label_save, result_dir, 'CRPS_MRMS_Baseline_2023.hdf')